In [1]:
import os
import re
import sys
import string
import random
from collections import namedtuple
from subprocess32 import call
from config import PATH_SAGE
from config import TEMP_DIR
import time
from datetime import datetime

def ordered_set(seq):
    """
    This function eliminates duplicated elements in a given list, 
    and keeping the order of appearance unchanged, 
    returns a list in which each elements appears only once
    """

    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def get_algebraic_equation(path):

    contents = None
    try:
        if os.path.isfile(path):
            with open(path, 'r') as fileobj:
                contents = fileobj.read()
    except (TypeError, ValueError):
        pass
    
    contents = contents.strip()
    problem_name = find_problem_name(contents)
    sections = split_contents_by_sections(remove_comments(contents))
    
    algebraic_relations = sections.get('algebraic relations', '')
    algebraic_equations_file = os.path.join(TEMP_DIR, 'algebraic_equations.txt')
    if algebraic_relations != '':   
        with open(algebraic_equations_file, 'w') as equations_file:
            equations_file.write(algebraic_relations)
    print(algebraic_equations_file)


def read_relation_file(path, preprocess=1, D=2, log=0):
    """
    Reads a relation file in GD format and parses it into a systems of connection relations
    """

#     rnd_string_tmp = '%030x' % random.randrange(16**30)
    contents = None
    try:
        if os.path.isfile(path):
            with open(path, 'r') as fileobj:
                contents = fileobj.read()
    except (TypeError, ValueError):
        pass

    contents = contents.strip()
    problem_name = find_problem_name(contents)
    sections = split_contents_by_sections(remove_comments(contents))

    connection_relations = sections.get('connection relations', '')
    algebraic_relations = sections.get('algebraic relations', '')
#     algebraic_equations_file = os.path.join(TEMP_DIR, 'algebraic_equations_%s.txt' % rnd_string_tmp)
    if algebraic_relations != '' and preprocess == 1:   
#         with open(algebraic_equations_file, 'w') as equations_file:
#             equations_file.write(algebraic_relations)
#         starting_time = time.time()
#         print('Preprocessing phase was started - %s' % datetime.now())
        macaulay_basis_file = os.path.join(TEMP_DIR, 'macaulay_basis.txt')
#         sage_process = call(["-python3", os.path.join("core", "macaulay.py"), 
#                              "-i", algebraic_equations_file,
#                              "-o", macaulay_basis_file,
#                              "-t", "degrevlex",
#                              "-D", str(D)],shell = 'Ture')
#         print(sage_process)
#         elapsed_time = time.time() - starting_time
#         print('Preprocessing phase was finished after %0.4f seconds' % elapsed_time)
        try:
            with open(macaulay_basis_file, 'r') as groebner_basis_file:
                groebner_basis = groebner_basis_file.read()
                temp = groebner_basis_file.readlines()[0:-2]
        except IOError:
            print(macaulay_basis_file + ' is not accessible!')
            sys.exit()
        algebraic_relations += '\n' + groebner_basis
        # algebraic_relations = groebner_basis
        if connection_relations == '':
            connection_relations = algebraic_relations_to_connection_relations(algebraic_relations.split('\n'))
        else:
            connection_relations += '\n' + algebraic_relations_to_connection_relations(algebraic_relations.split('\n'))
    elif algebraic_relations != '' and preprocess == 0:
        if connection_relations == '':
            connection_relations = algebraic_relations_to_connection_relations(algebraic_relations.split('\n'))
        else:
            connection_relations += '\n' + algebraic_relations_to_connection_relations(algebraic_relations.split('\n'))
    symmetric_relations, implication_relations, variables = parse_connection_relations(connection_relations)
    known_variables = sections.get('known', [])
    if known_variables != []:
        known_variables = known_variables.split('\n')
    known_variables.extend([rel[0] for rel in symmetric_relations if len(rel) == 1])
    known_variables = ordered_set(known_variables)
    symmetric_relations = [rel for rel in symmetric_relations if len(rel) != 1]
    target_variables = sections.get('target', [])
    if target_variables != []:
        target_variables = target_variables.split('\n')
    else:
        target_variables = variables
    weights_section = sections.get('weights', [])
    target_weights = None
    if weights_section != []:
        target_weights = dict()
        weights_section = weights_section.split('\n')        
        for element in weights_section:
            var, weight = element.split(' ')
            target_weights[var] = int(weight)
    notguessed_variables = sections.get('not guessed', [])
    if notguessed_variables != []:
        notguessed_variables = notguessed_variables.split('\n')
    parsed_data = {'problem_name': problem_name,
                   'variables': variables,
                   'known_variables': known_variables,
                   'target_variables': target_variables,
                   'target_weights' : target_weights,
                   'notguessed_variables': notguessed_variables,
                   'symmetric_relations': symmetric_relations,
                   'implication_relations': implication_relations}
#     if log == 0 and preprocess == 1:
#         os.remove(algebraic_equations_file)
#         os.remove(macaulay_basis_file)
    return parsed_data


def find_problem_name(contents):
    """
    Find the name of the problem by reading the first comments if it exists.
    If no comments are found 'gdproblem' is used by default.
    """

    pattern = re.compile(r"(?<=#).*?\n", re.DOTALL)
    match = pattern.search(contents)
    if match:
        return match.group().strip()
    return 'gdproblem'


def split_contents_by_sections(contents):
    """
    Split a relation file into multiple sections
    It searches for the following keywords and split the contents by section
    'connection relations', 'algebraic relations', 'known', 'target', 'not guessed'
    Raises ValueError when the given contents is not in a valid GD format    
    """

    keywords = {
        'connection relations': ('connection relation', 'connection relations'),
        'algebraic relations': ('algebraic relation', 'algebraic relations'),
        'known': ('known variables', 'known'),
        'target': ('target variables', 'target'),
        'not guessed': ('not guessed', ),
        'weights': ('weights', 'weight', ),
        'end': ('end', )
    }
    sections = []
    Section = namedtuple('Section', ('name', 'keyword_start', 'keyword_end'))

    for section_name, keywords in keywords.items():
        try:
            match, keyword_start, keyword_end = search_keywords(
                contents, keywords)
            sections.append(Section(section_name.lower(),
                                    keyword_start, keyword_end))
        except AttributeError:
            pass
    # sort by the start index of the section
    sections.sort(key=lambda x: x.keyword_start)

    if sections[-1].name != 'end':
        raise ValueError('File must end with an "end" keyword')

    parsed_sections = {}

    for i in range(len(sections) - 1):
        section = sections[i]
        next_section = sections[i + 1]
        parsed_sections[section.name] = contents[section.keyword_end:next_section.keyword_start].strip()

    return parsed_sections

def search_keywords(contents, keywords):
    """
    Search multiple keywords in a given contents case insensitively, and return the first match
    Raises ValueError when none of the keywords is found in the contents
    """

    sense_pattern = re.compile(
        '|'.join(rf'\b{re.escape(keyword)}\b' for keyword in keywords), re.IGNORECASE)
    match = sense_pattern.search(contents)
    return match.group(), match.start(), match.end()

def remove_comments(contents):
    """
    Remove the comments from the contents
    """
    
    contents = re.sub(re.compile(r"#.*?\n", re.DOTALL), "",
                      contents)  # remove all occurrences of #COMMENT from line
    return contents

def parse_connection_relations(connection_relations):
    variables = []
    symmetric_relations = []
    implication_relations = []
    connection_relations = connection_relations.replace(' ', '')
    connection_relations = connection_relations.split('\n')
    for rel in connection_relations:
        # Extract implication relations
        if '=>' in rel:
            rel = re.split(',|=>', rel)
            implication_relations.append(rel)
        # Extract symmetric relations
        else:
            rel = rel.split(',')
            symmetric_relations.append(rel)
        variables.extend(rel)
    variables = ordered_set(variables)
    return symmetric_relations, implication_relations, variables


def random_prefix_generator(N):
    """
    Generate a random string of length N
    """

    return ''.join(random.choice(string.ascii_uppercase) for _ in range(N))

def get_variables_from_monomial(monomial):
    """
    It is fed by a non-constant monomial like x*y*z and 
    returns a list consisting of given monomial's
    variables. which in this case are: ['x', 'y', 'z']
    """
    assert(not monomial.isdigit())
    temp = monomial.split('*')
    temp.sort()
    return temp

def get_variables_from_list_of_monomials(list_of_monomials):
    """
    It is fed by a list of monomials and returns the variables 
    which are used in the given monomials
    """
    vars = set()
    for monomial in list_of_monomials:
        vars = vars.union(get_variables_from_monomial(monomial))
    temp = list(vars)
    temp.sort()
    return temp

def get_monomials_from_polynomial(polynomial):
    """
    It is fed by a polynomial, and returns its monomials
    """

    monomials = polynomial.split('+')
    if '1' in monomials:
        monomials.remove('1')
    if '0' in monomials:
        monomials.remove('0')
    return list(set(monomials))

def get_monomials_from_list_of_polys(polys):
    """
    It is fed by a list of polynomials and returns
    all of the monomials which are used in the given 
    polynomials
    """

    monomials = set()
    for poly in polys:
        monos = set(get_monomials_from_polynomial(poly))
        monomials = monomials.union(monos)
    return list(monomials)

def degree_of_monomial(monomial):
    """
    Returns the degree of the given monomial
    """

    vars = get_variables_from_monomial(monomial)
    return len(vars)

def algebraic_relations_to_connection_relations(algebraic_relations):
    """
    Generate the connection relations derived from the given algebraic relations by introducing new variables
    It currently work for boolean polynomial relations merely
    """

    connection_relations = []
    if algebraic_relations[-1] == '':
        algebraic_relations[-1:] = []
    algebraic_relations = [poly.replace(' ', '') for poly in algebraic_relations]
    all_monomials = get_monomials_from_list_of_polys(algebraic_relations)
    algebraic_variables = get_variables_from_list_of_monomials(all_monomials)
    dummy_vars_prefix = random_prefix_generator(4)
    substitution_dictionary = dict()
    for monomial in all_monomials:
        if degree_of_monomial(monomial) >= 2:
            monomial_variables = get_variables_from_monomial(monomial)
            var_indices = [algebraic_variables.index(
                x) for x in monomial_variables]
            var_indices = list(map(str, var_indices))
            dummy_var = "{0}{1}".format(
                dummy_vars_prefix, "".join(var_indices))
            if dummy_var not in substitution_dictionary.values():
                connection_relations.append("{0}=>{1}".format(
                    ",".join(monomial_variables), dummy_var))
            substitution_dictionary[monomial] = dummy_var

    for poly in algebraic_relations:
        linearized_relation = [substitution_dictionary.get(term, term) for term in poly.split("+")]
        if '1' in linearized_relation:
            linearized_relation.remove('1')
        if '0' in linearized_relation:
            linearized_relation.remove('0')
        connection_relations.append(",".join(linearized_relation))
    return "\n".join(connection_relations)

In [2]:
# from core.parsesolution import parse_solver_solution
from gurobipy import *
# from core.inputparser import read_relation_file
import os
import time
import random
from core.parsesolution import parse_solver_solution
# from core.graphdrawer import draw_graph
# from config import TEMP_DIR

class ReduceGDtoMILP:
    """
    ReduceGDtoMILP-version4-kb
    This class reduces the guess-and-determine attack and 
    key-bridging to an MILP problem, and then solves it via Gurobi
    Created by Hosein Hadipour
    version4-kb
    July 13, 2020
    mod 0: Maximization
    mod 1: Minimization
    max_guess:  The number of guessed variables in mod 0,
                and an upper bound for the number of guessed variables in mod 1
    max_steps:  Number of state copies 
    """

    count = 0

    def __init__(self, inputfile_name='', outputfile_name='output', max_guess=0, max_steps=0, direction='min',\
        tikz=0, preprocess=1, D=2, dglayout="dot", log=0):
        self.inputfile_name = inputfile_name
        self.rnd_string_tmp = '%030x' % random.randrange(16**30)        
        self.output_dir = outputfile_name
        self.max_guess = max_guess
        self.max_steps = max_steps
        self.direction = direction
        self.dglayout = dglayout
        self.log = log
        ###############################
        # Read and parse the input file
        parsed_data = read_relation_file(self.inputfile_name, preprocess, D)
        self.problem_name = parsed_data['problem_name']
        self.variables = parsed_data['variables']
        self.known_variables = parsed_data['known_variables']
        self.target_variables = parsed_data['target_variables']
        self.target_weights = parsed_data['target_weights']
        self.notguessed_variables = parsed_data['notguessed_variables']
        self.symmetric_relations = parsed_data['symmetric_relations']
        self.implication_relations = parsed_data['implication_relations']
        self.num_of_relations = len(
            self.symmetric_relations) + len(self.implication_relations)
        self.num_of_vars = len(self.variables)
        ###############################
        if self.max_guess is None:
            self.max_guess = len(self.target_variables)
            print('The number of guessed variables is set to be at most %d' % self.max_guess)
        self.lpfile_name = 'milp_mg%d_ms%d_%s_%s.lp' % (
                            self.max_guess,\
                            max_steps, direction, self.rnd_string_tmp)
        self.deductions = self.generate_possible_deductions()
        # milp_variables is initialized to the variables corresponding to the initial state variables
        self.milp_variables = ['%s_0' % v for v in self.variables]
        self.time_limit = -1
        self.tikz = tikz

    def ordered_set(self, seq):
        """
        This method eliminates duplicated elements in a given list, 
        and returns a list in which each elements appears only once
        """

        seen = set()
        seen_add = seen.add
        return [x for x in seq if not (x in seen or seen_add(x))]

    def generate_possible_deductions(self):
        """
        This method generates all possible deductions 
        Core idea of information propagation: 
            If there is a relation in which all terms are known except one, 
            then the value of the last term can be determined as well. 
        """

        possible_deductions = {}
        for v in self.variables:
            possible_deductions[v] = [[v]]
            for rel in self.symmetric_relations:
                if v in rel:
                    temp = rel.copy()
                    temp.remove(v)
                    possible_deductions[v].append(temp)
            for rel in self.implication_relations:
                if v == rel[-1]:
                    temp = rel.copy()
                    temp.remove(v)
                    possible_deductions[v].append(temp)
        return possible_deductions

    def generate_objective_function(self):
        """
        This method generates the objective function of the MILP problem
        """

        if self.direction == 'max':
            objective_function = 'Maximize\n'
            final_state = ['%s_%d' % (v, self.max_steps)
                           for v in self.target_variables]
            objective_function += ' + '.join(final_state)
        elif self.direction == 'min':
            objective_function = 'Minimize\n'
            unknown_init_state_vars = [v for v in self.variables if v not in self.known_variables]
            if self.target_weights != None:
                wights = [self.target_weights.get(v, 1) for v in unknown_init_state_vars]
                weighted_init_state = zip(wights, unknown_init_state_vars)
                objective_list = ['%d %s_%d' % (w, v, 0) for (w, v) in weighted_init_state]                
            else:
                objective_list = ['%s_%d' % (v, 0) for v in unknown_init_state_vars]
            objective_function += ' + '.join(objective_list)
        objective_function += '\n'
        return objective_function

    def generate_initial_conditions(self):
        """
        This method generates the initial constraints consisting of 
        constrains corresponding to the maximum number of guessed variables, and 
        number of known variables in the initial state
        """

        initial_constraints = 'Subject To\n'

        unknown_vars = ['%s' % v for v in self.variables if v not in self.known_variables]
        unknown_init_state_vars = ['%s_0' % v for v in unknown_vars]
        if self.target_weights != None:
            LHS1 = ['%d %s_0' % (self.target_weights[v], v) for v in unknown_vars]
            LHS1 = ' + '.join(LHS1)
        else:
            LHS1 = ' + '.join(unknown_init_state_vars)
        RHS1 = self.max_guess
        final_state_target_vars = ['%s_%d' % (
            v, self.max_steps) for v in self.target_variables]
        LHS2 = ' + '.join(final_state_target_vars)
        RHS2 = len(final_state_target_vars)

        if self.direction == 'max':
            # initial_constraints += '%s = %d\n' % (LHS1, RHS1)
            initial_constraints += '%s <= %d\n' % (LHS1, RHS1)
        elif self.direction == 'min':
            initial_constraints += '%s <= %d\n' % (LHS1, RHS1)
            initial_constraints += '%s = %d\n' % (LHS2, RHS2)          

        for v in self.known_variables:
            if v != '':
                initial_constraints += '%s_0 = 1\n' % v

        # Limit the notguessed variables to be equal to 0 in the first step of knowledge propagation
        for v in self.notguessed_variables:
            if v != '':                
                initial_constraints += '%s_0 = 0;\n' % v        
        return initial_constraints
    

    def generate_milp_constraints(self):
        """
        This method generates the milp constraints corresponding to the 
        obtained deductions.
        """

        milp_constraints = ''
        for step in range(self.max_steps):
            for v in self.variables:
                v_old = '%s_%d' % (v, step)
                v_new = '%s_%d' % (v, step + 1)
                tau = len(self.deductions[v])
                v_path_variables = ['%s_%d_%d' %
                                    (v, step + 1, i) for i in range(tau)]
                self.milp_variables.extend([v_new] + v_path_variables)
                #####################################-State variable constraints-#####################################
                ######################################################################################################
                ######################################################################################################
                LHS = ' + '.join(v_path_variables)
                milp_constraints += '\ Constraints corresponding to the state variable %s:\n' % v_new
                if tau == 1:
                    milp_constraints += '%s - %s = 0\n' % (v_new, LHS)
                else:
                    milp_constraints += '-2 %s + %s >= -1\n' % (v_new, LHS)
                    milp_constraints += '%d %s - %s >= 0\n' % (
                        tau, v_new, LHS.replace('+', '-'))
                #####################################-Path variable constraints-######################################
                ######################################################################################################
                ######################################################################################################
                for i in range(tau // 2):
                    v_connected_variables = ['%s_%d' % (
                        var, step) for var in self.deductions[v][i]]
                    LHS = ' - '.join(v_connected_variables)
                    kapa = len(v_connected_variables)
                    milp_constraints += '\ Constraints corresponding to the path variable %s:\n' % v_path_variables[i]
                    if kapa == 1:
                        milp_constraints += '%s - %s = 0\n' % (
                            v_path_variables[i], LHS)
                    else:
                        milp_constraints += '%s - %s >= %d\n' % (
                            v_path_variables[i], LHS, 1 - kapa)
                        milp_constraints += '-%d %s + %s >= 0\n' % (
                            kapa, v_path_variables[i], LHS.replace('-', '+'))
                #####################################-Path variable constraints-######################################
                ######################################################################################################
                ######################################################################################################
                for i in range(tau // 2, tau):
                    v_connected_variables = ['%s_%d' % (
                        var, step) for var in self.deductions[v][i]]
                    LHS = ' - '.join(v_connected_variables)
                    kapa = len(v_connected_variables)
                    milp_constraints += '\ Constraints corresponding to the path variable %s:\n' % v_path_variables[i]
                    if kapa == 1:
                        milp_constraints += '%s - %s = 0\n' % (
                            v_path_variables[i], LHS)
                    else:
                        milp_constraints += '%s - %s >= %d\n' % (
                            v_path_variables[i], LHS, 1 - kapa)
                        milp_constraints += '-%d %s + %s >= 0\n' % (
                            kapa, v_path_variables[i], LHS.replace('-', '+'))
                ######################################################################################################
                ######################################################################################################
                ######################################################################################################
                # Given that the order of putting constraints into an LP file have a great impact on the performance of
                # Gurobi (and other CP solvers), user may want to change the order of constraints inside the generated
                # LP file in order to find a more suitable ordering. It is not known which odering might be the best from
                # the theoretical point of view in general. However, according to my experiences it may be better to put the connected
                # relations closer to each other. For example, change the order of the above blocks to see what impact it has
                # on the performance.
        return milp_constraints

    def declare_binary_variables(self):
        """
        This function declares the binary variables into the generated LP file
        """

        constraints = 'Binary\n'
        for v in self.milp_variables:
            constraints += '%s\n' % v
        return constraints

    def make_model(self):
        """
        This method makes the MILP model, and then write it into an LP file
        """

        print('Generating the MILP model ...')
        start_time = time.time()
        lp_str = self.generate_objective_function()
        lp_str += self.generate_initial_conditions()
        lp_str += self.generate_milp_constraints()
        lp_str += self.declare_binary_variables()
        lp_str += 'End\n'
        lp_file_path = os.path.join('lp', self.lpfile_name)
        elapsed_time = time.time() - start_time
        with open(lp_file_path, 'w') as lpfile:
                lpfile.write(lp_str) 
        if self.log == 1:
            print('MILP model was generated, and written into %s after %0.2f seconds' % (
                lp_file_path, elapsed_time))
        else:
            print('MILP model was generated after %0.2f seconds' % elapsed_time)

    def parse_and_write_solution(self):
        """
        This method parses the obtained soltuion (If MILP model is feasible), 
        and derives the determination flow such that user can simply find
        how the target variables are determined
        """
        
        self.vertices = []
        self.edges = []
        
        separator_line = ''.join(['#']*60)
        output_buffer = ''
        output_buffer += 'Number of relations: %d\n' % self.num_of_relations
        output_buffer += 'Number of variables: %d\n' % self.num_of_vars
        output_buffer += 'Number of target variables: %d\n' % len(
            self.target_variables)
        output_buffer += 'Number of known variables: %d\n' % len(
            self.known_variables)
        output_buffer += 'Number of guessed variables: %d\n' % len(
            self.guessed_vars)
        output_buffer += 'Number of state copies (max_steps): %d\n' % self.max_steps
        output_buffer += 'An upper bound for the number of guessed variables given by user (max_guess): %d\n' % self.max_guess
        output_buffer += '%d out of %d state variables are known after %d state copies\n' % (
            self.final_info, self.num_of_vars, self.max_steps)
        output_buffer += separator_line
        output_buffer += '\nThe following %d variable(s) are guessed:\n%s\n' % (
            len(self.guessed_vars), ', '.join(self.guessed_vars))
        output_buffer += separator_line
        output_buffer += '\nThe following %d variable(s) are initially known:\n%s\n' % (
            len(self.known_variables), ', '.join(self.known_variables))
        output_buffer += separator_line
        output_buffer += '\nTarget variables:\n%s\n' % ', '.join(
            self.target_variables)

        output_buffer += separator_line
        output_buffer += '\nDetermination flow:\n'
        step = 0
        determined_target_variables = dict(
            zip(self.target_variables, [0]*(len(self.target_variables))))
        for v in self.known_variables + self.guessed_vars:
            determined_target_variables[v] = 1
        while step < self.max_steps + 1 and not(all(determined_target_variables.values())):
            sub_title = '\nState %d:\n' % step
            output_buffer += sub_title
            for rel in self.symmetric_relations:
                rel_known_vars = [
                    v for v in rel if self.solutions[step]['%s_%d' % (v, step)] == 1]
                rel_str = ', '.join(rel)
                if len(rel_known_vars) == (len(rel) - 1):
                    determined_var = list(set(rel) - set(rel_known_vars))
                    lhs = ', '.join(rel_known_vars)
                    rhs = ', '.join(determined_var)
                    self.vertices.append(rhs)
                    for vr in rel_known_vars:
                        self.vertices.append(vr)
                        self.edges.append((vr, rhs))
                    if determined_var[0] in self.target_variables:
                        determined_target_variables[determined_var[0]] = 1
                    output = '%s in symmetric relation [%s] are known: %s ===> %s' % (
                        lhs, rel_str, lhs, rhs) + '\n'
                    output_buffer += output
            for rel in self.implication_relations:
                rel_known_vars = [
                    v for v in rel if self.solutions[step]['%s_%d' % (v, step)] == 1]
                rel_str = ', '.join(rel)
                if rel_known_vars == rel[0:-1]:
                    determined_var = [rel[-1]]
                    lhs = ', '.join(rel_known_vars)
                    rhs = ', '.join(determined_var)
                    self.vertices.append(rhs)
                    for vr in rel_known_vars:
                        self.vertices.append(vr)
                        self.edges.append((vr, rhs))
                    if determined_var[0] in self.target_variables:
                        determined_target_variables[determined_var[0]] = 1
                    output = '%s in implication relation [%s] are known: %s ===> %s' % (
                        lhs, rel_str, lhs, rhs) + '\n'
                    output_buffer += output
            output_buffer += separator_line
            step += 1
        self.finally_known = [v for v in self.variables if self.solutions[self.max_steps]['%s_%d' % (v, self.max_steps)] == 1]
        output_buffer += '\nThe following variables are known in final state:\n%s' % ', '.join(self.finally_known)
        with open(self.output_dir, 'w') as outputfile_obj:
            outputfile_obj.write(output_buffer)

    def solve_model(self):
        """
        This method uses Gurobi to solve the obtained MILP problem
        """
        lpfile_path = os.path.join('lp', self.lpfile_name)
        self.milp_model = read(lpfile_path)
        if self.log == 0:
            os.remove(lpfile_path)
        if self.time_limit != -1:
            self.milp_model.params.TimeLimit = self.time_limit
        # 0 (default: Gurobi strikes a balance between finding feasible solution and proving the optimality)
        self.milp_model.params.MIPFocus = 0
        # 1 (quickly find a feasible solution)
        # 2 (proving optimality is in priority)
        # 3 (focus on the best bound)
        self.milp_model.params.Threads = 0  # 0 (default value)
        self.milp_model.params.OutputFlag = 1  # 1 (default value)

        start_time = time.time()
        ##########################
        ##########################
        self.milp_model.optimize()
        ##########################
        ##########################
        elapsed_time = time.time() - start_time
        print('Solving process was finished after %0.2f seconds' % elapsed_time)

        if self.milp_model.Status == GRB.OPTIMAL or self.milp_model.Status == GRB.INTERRUPTED or self.milp_model.Status == GRB.TIME_LIMIT:
            if self.milp_model.SolCount == 0:
                print(
                    'No solution found! Perhaps, I need more time to solve this problem.')
                return
            self.objval = self.milp_model.objval
            self.solutions = [0]*(self.max_steps + 1)
            for step in range(self.max_steps + 1):
                state_vars = ['%s_%d' % (v, step) for v in self.variables]
                state_values = list(
                    map(lambda x: int(self.milp_model.getVarByName(x).Xn), state_vars))
                self.solutions[step] = dict(zip(state_vars, state_values))
            if self.milp_model.SolCount == 0:
                print('Sorry! There is no solution to be parsed.\nTry again please.')
                return
            else:
                if self.log == 1:
                    gurobi_solution_file_name = 'grbsol_mg%d_ms%d_%s.sol' % (
                        self.max_guess, self.max_steps, self.direction)
                    grb_solution_file_path = os.path.join(
                        TEMP_DIR, gurobi_solution_file_name)
                    self.milp_model.write(grb_solution_file_path)                    
                parse_solver_solution(self)
#                 draw_graph(self.vertices, self.edges, self.known_variables, self.guessed_vars,\
#                     self.output_dir, self.tikz, self.dglayout)
        elif self.milp_model.Status == GRB.INFEASIBLE:
            print('The obtained milp model is infeasible')
        else:
            print('Unknown error!')
        return 'The following %d variable(s) are guessed:\n%s\n' % (
            len(self.guessed_vars), ', '.join(self.guessed_vars))

In [3]:
get_algebraic_equation('relationfile_lblock_22r.txt')

temp\algebraic_equations.txt


In [5]:
gdmilp = ReduceGDtoMILP(inputfile_name='relationfile_lblock_22r.txt',
                        outputfile_name='Autoguess_Lblock',
                        max_guess=73,
                        max_steps=25,
                        direction='max')
gdmilp.make_model()
gdmilp.solve_model()

Generating the MILP model ...
MILP model was generated after 32.49 seconds
Read LP format model from file lp\milp_mg73_ms25_max_e2783a3b941e1838bdc17d72752b5b.lp
Reading time = 0.92 seconds
: 314001 rows, 261040 columns, 1017040 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 314001 rows, 261040 columns and 1017040 nonzeros
Model fingerprint: 0x2949c97f
Variable types: 0 continuous, 261040 integer (261040 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 266275 rows and 240913 columns (presolve time = 5s) ...
Presolve removed 268022 rows and 241796 columns
Presolve time: 7.09s
Presolved: 45979 rows, 19244 columns, 174505 nonzeros
Variable types: 0 continuous, 19244 integer (19244 binary)

Det

  3821  1293  144.00000   36  166  143.00000  144.00000  0.70%   434  953s
  3825  1300  144.00000   37  252  143.00000  144.00000  0.70%   434  966s
  3833  1310  144.00000   38  191  143.00000  144.00000  0.70%   434  974s
  3847  1320  144.00000   39  406  143.00000  144.00000  0.70%   434  978s
  3875  1338  144.00000   40  259  143.00000  144.00000  0.70%   434  982s
  3889  1349  144.00000   40  322  143.00000  144.00000  0.70%   434  985s
  3924  1385  144.00000   42  265  143.00000  144.00000  0.70%   442  993s
  3952  1418  144.00000   43  292  143.00000  144.00000  0.70%   450 1001s
H 3954  1352                     144.0000000  144.00000  0.00%   450 1001s

Cutting planes:
  Gomory: 45
  Lift-and-project: 32
  Cover: 4
  Implied bound: 2487
  Clique: 906
  Flow cover: 70
  GUB cover: 5
  Zero half: 372
  RLT: 181
  BQP: 51

Explored 3994 nodes (3757962 simplex iterations) in 1002.05 seconds (513.66 work units)
Thread count was 20 (of 20 available processors)

Solution count 1

AttributeError: 'ReduceGDtoMILP' object has no attribute 'equal_relations'

In [4]:
gdmilp = ReduceGDtoMILP(inputfile_name='relationfile_lblock_22r.txt',
                        outputfile_name='Autoguess_Lblock',
                        max_guess=144,
                        max_steps=25,
                        direction='min')
gdmilp.make_model()
gdmilp.solve_model()

Generating the MILP model ...
MILP model was generated after 30.67 seconds
Read LP format model from file lp\milp_mg144_ms25_min_b76c3de77a3f94de29c754ca5e98dd.lp
Reading time = 0.99 seconds
: 314002 rows, 261040 columns, 1017184 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 314002 rows, 261040 columns and 1017184 nonzeros
Model fingerprint: 0x2efd4749
Variable types: 0 continuous, 261040 integer (261040 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 268307 rows and 242518 columns (presolve time = 5s) ...
Presolve removed 276830 rows and 245293 columns
Presolve time: 9.70s
Presolved: 37172 rows, 15747 columns, 141479 nonzeros
Variable types: 0 continuous, 15747 integer (15747 binary)

Deterministic concurrent LP optimizer: primal and

   695   627   13.12830   46 7737   76.00000    1.51323  98.0%  6062 1441s
   752   666   15.10246   50 7710   76.00000    1.51323  98.0%  5875 1473s
   793   730   15.11552   54 7706   76.00000    1.51323  98.0%  5781 1506s
   859   806   15.15266   57 7714   76.00000    1.51323  98.0%  5525 1534s
   941   870   16.08091   64 7602   76.00000    1.51323  98.0%  5290 1560s
  1010   999   16.20542   65 7553   76.00000    1.51323  98.0%  5045 1585s
  1143  1099   16.50247   67 7612   76.00000    1.51323  98.0%  4638 1618s
H 1236  1099                      74.0000000    1.51323  98.0%  4428 1618s
  1253  1207   17.19374   69 7547   74.00000    1.51323  98.0%  4375 1641s
  1363  1331   19.71413   73 7263   74.00000    1.51323  98.0%  4162 1668s
  1497  1450   19.40729   77 7304   74.00000    1.51323  98.0%  3944 1701s
  1656  1538    3.61514    6 9046   74.00000    1.51323  98.0%  3713 1743s
  1772  1669    5.75715    7 8914   74.00000    1.51323  98.0%  3612 1776s
  1925  1670   55.06897  

 14046 10228   64.92011   60 3702   73.00000   54.44948  25.4%  1050 2714s
 14546 10549   57.13673   56 3275   73.00000   54.46484  25.4%  1037 2736s
 15006 10843   56.41839   53 4346   73.00000   54.47496  25.4%  1028 2787s
 15385 11296   63.85518   67 3737   73.00000   54.59519  25.2%  1017 2810s
 15940 11654   58.54342   48 4047   73.00000   54.62884  25.2%  1004 2837s
 16370 12057   64.87606   57 3509   73.00000   54.63945  25.2%  1000 2861s
 16937 12525   61.01935   57 3897   73.00000   54.73520  25.0%   989 2888s
 17551 12884   57.86603   51 4252   73.00000   54.80644  24.9%   975 2917s
 18436 13304   56.69789   47 4074   73.00000   54.83590  24.9%   952 2944s
 19050 13794   61.02036   51 3521   73.00000   54.87044  24.8%   944 2973s
 19685 13795   66.49029   75 9903   73.00000   54.87044  24.8%   936 3056s
 19687 13796   62.76853   80 1675   73.00000   54.87044  24.8%   936 3065s
 19689 13798   68.78150   82 2758   73.00000   54.87044  24.8%   936 3070s
 19691 13799   67.30019  

 30761 18642   69.63268   80 2326   73.00000   61.24751  16.1%   874 3880s
 31217 18829   64.21233   59 3789   73.00000   61.25169  16.1%   872 3899s
 31658 19092   67.86496   69 2788   73.00000   61.25523  16.1%   872 3920s
 32187 19281   62.37004   60 4167   73.00000   61.26731  16.1%   869 3940s
 32665 19512   68.71141   75 3614   73.00000   61.28311  16.1%   868 3962s
 33199 19691   66.71014   67 3549   73.00000   61.39664  15.9%   866 3984s
 33726 19892   70.41150   83 2421   73.00000   61.44264  15.8%   865 4007s
 34243 20128   71.59298   85 2013   73.00000   61.48943  15.8%   865 4030s
 34806 20363   64.10775   62 3750   73.00000   61.54206  15.7%   864 4055s
 35412 20527   63.80609   59 3924   73.00000   61.56412  15.7%   862 4080s
 35979 20751   71.08626   83 3013   73.00000   61.59280  15.6%   863 4106s
 36527 21202   66.72315   69 3309   73.00000   61.62170  15.6%   865 4134s
 37282 21366   67.45401   68 3711   73.00000   61.62372  15.6%   861 4161s
 37840 21893   66.92343  

 59058 28722   68.93612   71 4222   73.00000   68.93612  5.57%   833 6552s
 59059 28722   71.07815   75 4150   73.00000   68.93739  5.57%   833 6561s
 59060 28723   71.16707   91 4178   73.00000   68.93755  5.56%   833 6570s
 59061 28724   71.22284   87 4150   73.00000   68.94099  5.56%   833 6578s
 59062 28724   70.04633   81 4190   73.00000   68.94120  5.56%   833 6587s
 59063 28725   68.94128   68 4126   73.00000   68.94128  5.56%   833 6597s
 59064 28726   70.55740   79 4179   73.00000   68.94128  5.56%   833 6605s
 59065 28726   68.94129   75 4152   73.00000   68.94129  5.56%   833 6614s
 59066 28727   68.94137   64 4186   73.00000   68.94137  5.56%   833 6624s
 59067 28728   71.68720   81 4192   73.00000   68.94550  5.55%   833 6634s
 59068 28728   68.94835   78 4166   73.00000   68.94835  5.55%   833 6642s
 59069 28729   68.94921   62 4197   73.00000   68.94921  5.55%   833 6652s
 59070 28730   71.41851   74 4223   73.00000   68.95071  5.55%   833 6661s
 59071 28730   69.24916  

 59179 28802   70.94476   85 2941   73.00000   69.53982  4.74%   831 7493s
 59182 28804   70.17872   75 3035   73.00000   69.56428  4.71%   831 7501s
 59183 28805   70.61357   75 2954   73.00000   69.56925  4.70%   831 7507s
 59186 28807   69.61127   60 3025   73.00000   69.61127  4.64%   831 7516s
 59187 28808   69.62000   64 2762   73.00000   69.62000  4.63%   831 7525s
 59188 28808   69.62104   65 2966   73.00000   69.62104  4.63%   831 7533s
 59189 28809   69.62118   69 2979   73.00000   69.62118  4.63%   831 7542s
 59190 28810   71.62169   92 2959   73.00000   69.62155  4.63%   831 7549s
 59191 28810   69.62351   69 2975   73.00000   69.62351  4.63%   831 7557s
 59192 28811   69.62801   77 2979   73.00000   69.62801  4.62%   831 7565s
 59193 28812   69.63420   73 2959   73.00000   69.63420  4.61%   831 7574s
 59194 28812   69.63525   72 3005   73.00000   69.63525  4.61%   831 7582s
 59195 28813   69.63576   72 2807   73.00000   69.63576  4.61%   831 7590s
 59196 28814   69.63651  

AttributeError: 'ReduceGDtoMILP' object has no attribute 'equal_relations'